We have green, orange and red lists

* The idea is to make use of most of orange word lists

* Making an entropy graph of our texts, and then putting those words in red lists where the other tokens also have a good probability getting selected in the texts, once a bias is added in them. But if text is short then this can lead to problem which in our case is not as we are dealing with review

* If a text is paraphrased then it will be having less orange words, whereas both in human and main ai will be having more orange words, from there by making some threshold we can figure out whether the text is paraphrased or not. 

* Taking care whether the quality of the text is getting reduced or not

In [2]:
import sys
sys.path.append("/DATA/priyanshu_2021cs26/zMohit/PeerDetection/code")

In [ ]:
# myTorch                  /DATA/priyanshu_2021cs26/anaconda3/envs/myTorch

In [1]:
from tester import *
from learning import *

/DATA/priyanshu_2021cs26/anaconda3/envs/myTorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /DATA/priyanshu_2021cs26/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /DATA/priyanshu_2021cs26/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /DATA/priyanshu_2021cs26/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /DATA/priyanshu_2021cs26/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
args = parse_args()
# print(args)
# if args.ppl:
#     testppl(args)
# else:
#     main(args)
args.model_name_or_path = "meta-llama/Llama-3.2-3B-Instruct"
args.max_new_tokens = 2000
args.min_new_tokens = 5
args.gamma = 0.9
args.attack_ep = 0.1


# Configurations that can be changed
args.delta = 1.0
checkpoint_file = f"checkpoints/check{args.delta}.json"
output_file = f"checkpoints/final{args.delta}.json"

device = torch.device('cuda')
# device = torch.device('cpu')
model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path, torch_dtype=torch.float16)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

# Load existing progress if checkpoint exists
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        processed_data = json.load(f)
    processed_titles = {entry["title"] for entry in processed_data}
else:
    processed_data = []
    processed_titles = set()

if run_all==True:
    read_file = read_json.read_old()
else:
    read_file = read_json.data

100%|██████████| 546/546 [00:00<00:00, 2764.82it/s]


In [3]:
titles = json.load(open("delta_test_titles.json"))
read_file = [read_file[i] for i in range(len(read_file)) if read_file[i]['title'] in titles]

In [23]:
titles[1], read_file[1]['title']

('LANGUAGE-INFORMED VISUAL CONCEPT LEARNING',
 'LANGUAGE-INFORMED VISUAL CONCEPT LEARNING')

In [25]:
n = 1
each_dic = read_file[n]
title = each_dic['title']

abstract = each_dic['abstract']
paper_text = each_dic['paper_text']

paper_content = f'''
title: {title}
abstract: {abstract}
paper text: {paper_text}
'''

content = f''' The peer review format and length should be of standard conference. \\
title: {title} \\
abstract: {abstract} \\
paper text: {paper_text} \\

Steps to follow :- \\
Step 1: Read the paper critically and Only write peer review and nothing else \\
Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
Step 2: Output Format: You must Return the Review enclosed between $$$\\ 
'''

system_prompt = ''' You are a Research Scientist. Your task is to thoroughly and critically read the paper and write a peer review of it.  Steps to follow :- \\
Step 1: Read the paper critically and Only write peer review and nothing else \\
Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
Step 2: Output Format: You must Return the Review enclosed between $$$'''

input_text = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": content},
]

prompt = input_text

In [26]:
title = titles[n]

In [27]:
watermark_processor = WatermarkLogitsProcessor_with_preferance(title=title,
                                                                vocab=list(tokenizer.get_vocab().values()),
                                                                args=args
                                                                )
def token_returner(text):
    # tokd_input = tokenizer.apply_chat_template(prompt, return_tensors="pt", add_generation_prompt=True, truncation=True,
    #                         max_length=args.prompt_max_length, return_dict=True).to(device)

    # tokd_input = {k: v.to(model.device) for k, v in tokd_input.items()}

    # if isinstance(tokd_input, torch.Tensor):
    #     tokd_input = {"input_ids": tokd_input}

    # tokens = tokd_input['input_ids'][0].tolist()
    tokens = tokenizer(text)
    g,r = watermark_processor._get_greenlist_ids(watermark_processor.title)
    return tokens, g, r

In [28]:
yup = json.load(open("peer_review_outputs_paraphrasing_attack.json"))
for x in yup:
    if x['title']==titles[n]:
        para = x

yup = json.load(open("peer_review_outputs_new.json"))
for x in yup:
    if x['title']==titles[n]:
        revw = x

In [29]:
orange,_,_ = token_returner(paper_content)

In [30]:
token_with, g_with, r_with = token_returner(revw['peer_review_with_watermark'])
token_without, g_without, r_without = token_returner(revw['peer_review_without_watermark'])

In [31]:
token_with_p, g_with_p, r_with_p = token_returner(para['peer_review_with_watermark'])
token_without_p, g_without_p, r_without_p = token_returner(para['peer_review_without_watermark'])

In [32]:
f = lambda a:1 if a in g_with else 0
[f(a) for a in token_with['input_ids']]

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [91]:
len(set(orange['input_ids']).intersection(token_with_p['input_ids']))

108

In [4]:
def delta_variation():
    for each_dic in tqdm(test_file):
        title = each_dic['title']
        if title in processed_titles:
            print(f"Skipping already processed: {title}")
            continue  # Skip already processed entries

        abstract = each_dic['abstract']
        paper_text = each_dic['paper_text']

        content = f''' The peer review format and length should be of standard conference. \\
        title: {title} \\
        abstract: {abstract} \\
        paper text: {paper_text} \\
        
        Steps to follow :- \\
        Step 1: Read the paper critically and Only write peer review and nothing else \\
        Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
        Step 2: Output Format: You must Return the Review enclosed between $$$\\ 
        '''

        system_prompt = ''' You are a Research Scientist. Your task is to thoroughly and critically read the paper and write a peer review of it.  Steps to follow :- \\
        Step 1: Read the paper critically and Only write peer review and nothing else \\
        Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
        Step 2: Output Format: You must Return the Review enclosed between $$$'''

        input_text = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content},
        ]

        # Generate outputs
        input_token_num, output_token_num, _, _, decoded_output_without_watermark, decoded_output_with_watermark, watermark_processor, _ = generate(
            input_text,
            args,
            model=model,
            device=device,
            tokenizer=tokenizer,
            title=title
        )
        ##Attack
        if args.attack_ep==0:
            pass
        else:
            #decoded_output_with_watermark,skip=attack_process(decoded_output_with_watermark,epsilon=args.attack_ep)
            #decoded_output_with_watermark=attack_run(decoded_output_with_watermark)
            decoded_output_with_watermark=response(decoded_output_with_watermark)
            
            # if not skip:
            #         print("Attacked Output:", attacked_output)
            # else:
            #         print("Attack skipped.")
            #         break

        
        # Detect scores for watermarked output
        if args.detect_mode == 'normal':
            gr_score_list = []
            max_sim = 0
            max_sim_idx = -1
            
            output_dict_with, gr_score_with, mark = detect(
                decoded_output_with_watermark,
                args,
                device=device,
                tokenizer=tokenizer,
                title=title
            )
            output_dict_without, gr_score_without, mark = detect(
                decoded_output_without_watermark,
                args,
                device=device,
                tokenizer=tokenizer,
                title=title
            )

        # Append the results to the list
        result = {
            "title": title,
            "abstract": abstract,
            "peer_review_without_watermark": decoded_output_without_watermark,
            "peer_review_with_watermark": decoded_output_with_watermark,
            "gr_score_with": safe_serialize(gr_score_with),  # Serialize safely
            "gr_score_without": safe_serialize(gr_score_without),  # Serialize safely
            "output_without": safe_serialize(output_dict_without),
            "output_with": safe_serialize(output_dict_with)
        }

        processed_data.append(result)
        processed_titles.add(title)

        # Save checkpoint after each iteration
        with open(checkpoint_file, "w") as f:
            json.dump(processed_data, f, indent=4)

        print(f"Processed: {title}")

    # Save the final results to the output file
    with open(output_file, "w") as f:
        json.dump(processed_data, f, indent=4)

    # Remove the checkpoint file once done
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)

    print(f"Saved results to {output_file}")

In [1]:
float(6)

6.0

## Don't see below 👇

In [4]:
each_dic = read_file[0]
title = each_dic['title']
if title in processed_titles:
    print(f"Skipping already processed: {title}")

abstract = each_dic['abstract']
paper_text = each_dic['paper_text']

content = f''' The peer review format and length should be of standard conference. \\
title: {title} \\
abstract: {abstract} \\
paper text: {paper_text} \\

Steps to follow :- \\
Step 1: Read the paper critically and Only write peer review and nothing else \\
Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
Step 2: Output Format: You must Return the Review enclosed between $$$\\ 
'''

system_prompt = ''' You are a Research Scientist. Your task is to thoroughly and critically read the paper and write a peer review of it.  Steps to follow :- \\
Step 1: Read the paper critically and Only write peer review and nothing else \\
Step 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\
Step 2: Output Format: You must Return the Review enclosed between $$$'''

input_text = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": content},
]

Skipping already processed: CAUSAL MODELLING AGENTS: CAUSAL GRAPH DIS-


In [9]:
# input_token_num, output_token_num, _, _, decoded_output_without_watermark, decoded_output_with_watermark, watermark_processor, _ = generate(
#             input_text,
#             args,
#             model=model,
#             device=device,
#             tokenizer=tokenizer,
#             title=title
#         )

prompt = input_text
gen_kwargs = dict(max_new_tokens=args.max_new_tokens,min_new_tokens=args.min_new_tokens)

watermark_processor = WatermarkLogitsProcessor_with_preferance(title=title,
                                                                vocab=list(tokenizer.get_vocab().values()),
                                                                args=args
                                                                )
watermark_processor.delta = args.delta

In [ ]:
# Generate outputs
input_token_num, output_token_num, _, _, decoded_output_without_watermark, decoded_output_with_watermark, watermark_processor, _ = generate(
    input_text,
    args,
    model=model,
    device=device,
    tokenizer=tokenizer,
    title=title
)
##Attack
if args.attack_ep==0:
    pass
else:
    #decoded_output_with_watermark,skip=attack_process(decoded_output_with_watermark,epsilon=args.attack_ep)
    #decoded_output_with_watermark=attack_run(decoded_output_with_watermark)
    decoded_output_with_watermark=response(decoded_output_with_watermark)
    
    # if not skip:
    #         print("Attacked Output:", attacked_output)
    # else:
    #         print("Attack skipped.")
    #         break


# Detect scores for watermarked output
if args.detect_mode == 'normal':
    gr_score_list = []
    max_sim = 0
    max_sim_idx = -1
    
    output_dict_with, gr_score_with, mark = detect(
        decoded_output_with_watermark,
        args,
        device=device,
        tokenizer=tokenizer,
        title=title
    )
    output_dict_without, gr_score_without, mark = detect(
        decoded_output_without_watermark,
        args,
        device=device,
        tokenizer=tokenizer,
        title=title
    )

# Append the results to the list
result = {
    "title": title,
    "abstract": abstract,
    "peer_review_without_watermark": decoded_output_without_watermark,
    "peer_review_with_watermark": decoded_output_with_watermark,
    "gr_score_with": safe_serialize(gr_score_with),  # Serialize safely
    "gr_score_without": safe_serialize(gr_score_without),  # Serialize safely
    "output_without": safe_serialize(output_dict_without),
    "output_with": safe_serialize(output_dict_with)
}

processed_data.append(result)
processed_titles.add(title)

# Save checkpoint after each iteration
with open(checkpoint_file, "w") as f:
    json.dump(processed_data, f, indent=4)

print(f"Processed: {title}")

# Save the final results to the output file
with open(output_file, "w") as f:
    json.dump(processed_data, f, indent=4)

# Remove the checkpoint file once done
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)

print(f"Saved results to {output_file}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


## Don't see below this 👇

In [10]:
if args.use_sampling:
    gen_kwargs.update(dict(
        do_sample=True, 
        top_k=0,
        temperature=args.sampling_temp
    ))
else:
    gen_kwargs.update(dict(
        num_beams=args.n_beams
    ))
generate_without_watermark = partial(
    model.generate,
    **gen_kwargs
)

generate_with_watermark = partial(
    model.generate,
    logits_processor=LogitsProcessorList([watermark_processor]),
    return_dict_in_generate=True, 
    output_scores=True,
    **gen_kwargs
)

if args.prompt_max_length:
    pass
elif hasattr(model.config,"max_position_embedding"):
    args.prompt_max_length = model.config.max_position_embeddings-args.max_new_tokens
else:
    args.prompt_max_length = 2048-args.max_new_tokens

In [6]:
args.model_name_or_path

'meta-llama/Llama-3.2-3B-Instruct'

In [21]:
prompt

[{'role': 'system',
  'content': ' You are a Research Scientist. Your task is to thoroughly and critically read the paper and write a peer review of it.  Steps to follow :- \\\nStep 1: Read the paper critically and Only write peer review and nothing else \\\nStep 2: In Peer review Only write Paper Summary, Strengths, Weaknesses, Suggestions for Improvement and Recommendation \\\nStep 2: Output Format: You must Return the Review enclosed between $$$'},
 {'role': 'user',
  'content': ' The peer review format and length should be of standard conference. \\\ntitle: CAUSAL MODELLING AGENTS: CAUSAL GRAPH DIS- \\\nabstract: Centre for Medical Image Computing, UCL, London, United Kingdom Microsoft Research, Cambridge ABSTRACT Scientific discovery hinges on the effective integration of metadata, which refers to a set of conceptual operations such as determining what information is relevant for inquiry, and data, which encompasses physical operations such as observation and experimentation. This

In [11]:
tokd_input = tokenizer.apply_chat_template(prompt, return_tensors="pt", add_generation_prompt=True, truncation=True,
                        max_length=args.prompt_max_length, return_dict=True).to(device)

tokd_input = {k: v.to(model.device) for k, v in tokd_input.items()}

if isinstance(tokd_input, torch.Tensor):
    tokd_input = {"input_ids": tokd_input}

In [7]:
from watermark_processor import *

In [9]:

class yupWatermark(WatermarkBase, LogitsProcessor):
    def __init__(self, title: str, **kwargs):
        self.title = title
        self.delta: float = 2.0
        self.decrease_delta: bool = True,
        self.idx_t = 0
        super().__init__(**kwargs)

    def __call__(self, input_ids: torch.Tensor, scores: torch.FloatTensor) ->torch.FloatTensor:
        global yupScore
        yupScore = scores
        return scores

In [19]:
len(tokd_input['input_ids'][0])

48

In [14]:
prompt = [{"role":"user",
           "content":"Are you working?"}]
tokd_input = tokenizer.apply_chat_template(prompt, return_tensors="pt", add_generation_prompt=True, truncation=True,
                        max_length=args.prompt_max_length, return_dict=True).to(device)

tokd_input = {k: v.to(model.device) for k, v in tokd_input.items()}

if isinstance(tokd_input, torch.Tensor):
    tokd_input = {"input_ids": tokd_input}

In [9]:
model.generate(**tokd_input, max_new_tokens=2000,min_new_tokens=5, pad_token_id=tokenizer.eos_token_id)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1114,   4448,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  11787,    499,   3318,     30, 128009, 128006,
          78191, 128007,    271,     40,   2846,   2561,    323,   5644,    311,
           7945,    499,     13,   2650,    649,    358,   1520,    499,   3432,
             30, 128009]])

In [15]:
model.generate(**tokd_input, max_new_tokens=2000,min_new_tokens=5, pad_token_id=tokenizer.eos_token_id)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1114,   4448,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  11787,    499,   3318,     30, 128009, 128006,
          78191, 128007,    271,     40,   2846,    264,   3544,   4221,   1646,
             11,    779,    358,   2846,   2744,    330,    263,      1,    323,
           5644,    311,   7945,    499,     13,    358,   1541,    956,   1397,
           6212,    477,  18808,     11,    323,    358,   2846,   2561,    220,
           1187,     14,     22,    311,   4320,    701,   4860,    323,   3493,
           2038,    389,    264,   7029,   2134,    315,  13650,     13,   2650,
            649,    358,   1520,    499,   3432,     30, 128009]])

In [10]:
model.generate(**tokd_input, **gen_kwargs, pad_token_id=tokenizer.eos_token_id)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [12]:
generate_with_watermark = partial(
        model.generate,
        logits_processor=LogitsProcessorList([watermark_processor]),
        return_dict_in_generate=True, 
        output_scores=True,
        **gen_kwargs
    )
out = generate_with_watermark(**tokd_input)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
torch.manual_seed(args.generation_seed)
out = generate_with_watermark(**tokd_input)
torch.manual_seed(args.generation_seed)
output_with_watermark = out[0]
decoded_output_with_watermark = tokenizer.batch_decode(output_with_watermark, skip_special_tokens=True)[0]

truncation_warning = True if tokd_input["input_ids"].shape[-1] == args.prompt_max_length else False
redecoded_input = tokenizer.batch_decode(tokd_input["input_ids"], skip_special_tokens=True)[0]

torch.manual_seed(args.generation_seed)
output_without_watermark = generate_without_watermark(**tokd_input)
decoded_output_without_watermark = tokenizer.batch_decode(output_without_watermark, skip_special_tokens=True)[0]

/DATA/priyanshu_2021cs26/anaconda3/envs/myTorch/lib/python3.11/site-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (7925) is larger than the maximum possible length (7850). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 7850, its default value.
  warnings.warn(
/DATA/priyanshu_2021cs26/anaconda3/envs/myTorch/lib/python3.11/site-packages/transformers/generation/utils.py:1407: UserWarning: Unfeasible length constraints: `min_new_tokens` (100), when added to the prompt length (7825), is larger than the maximum possible length (7850). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 7850, its default value.
  warnings.warn(
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [519,0,0], th

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 